In [ ]:
import yfinance as yf 
import time
import datetime
import pandas as pd
import sqlite3

In [ ]:
month_symbol_mapper = {
    'jul':'N',
    'aug':'Q',
    'sep':'U',
    'oct':'V',
    'nov':'X',
    'dec':'Z',
    'jan':'F',
    'feb':'G',
    'mar':'H',
    'apr':'J',
    'may':'K',
    'jun':'M',
}

def oil_futures_ticker_generator(year='22',month='jan'):
    ticker =  f"CL{month_symbol_mapper[month]}{year}.NYM"
    print(ticker)
    return ticker

def get_yahoo_data(ticker="CLN22.NYM",start=None,end=None):
    yahoo_ticker_interface = yf.Ticker(ticker)
    if start is None or end is None:        
        price_data = yahoo_ticker_interface.history(period='max')
    else:
        price_data = yahoo_ticker_interface.history(start=start,end=end)
    price_data.drop(labels=['Dividends','Stock Splits'], axis=1, inplace=True)
    price_data['Ticker']=ticker
    return price_data

def get_front_month():
    month_string = datetime.datetime(datetime.date.today().year,datetime.date.today().month+2,1).strftime('%b').lower()
    month_number = datetime.datetime(datetime.date.today().year,datetime.date.today().month+2,1).strftime('%-m')
    return {'month_string': month_string, 'month_number':month_number}

def get_back_month():
    month_string = datetime.datetime(datetime.date.today().year,datetime.date.today().month+3,1).strftime('%b').lower()
    month_number = datetime.datetime(datetime.date.today().year,datetime.date.today().month+3,1).strftime('%-m')
    return {'month_string': month_string, 'month_number':month_number}

def get_current_cycle():
    cycle_start = datetime.datetime(datetime.date.today().year,datetime.date.today().month,1)
    today = datetime.date.today()
    return {'cycle_start':cycle_start, 'today':today}


In [ ]:
front_month = get_front_month()
back_month = get_back_month()
front_month_ticker = oil_futures_ticker_generator(month= front_month['month_string'])
back_month_ticker = oil_futures_ticker_generator(month= back_month['month_string'])
current_cycle = get_current_cycle()
front_month_data = get_yahoo_data(ticker=front_month_ticker,start=current_cycle['cycle_start'],end=current_cycle['today'])
back_month_data = get_yahoo_data(ticker=back_month_ticker,start=current_cycle['cycle_start'],end=current_cycle['today'])


In [ ]:
front_month_data.head()


In [ ]:
back_month_data.head()

In [ ]:
merged = pd.merge(front_month_data, back_month_data, left_index=True, right_index=True, how='inner', suffixes=('_front','_back'))
merged['Time_Spread']= merged['Close_back']-merged['Close_front']
merged

In [ ]:
oil = yf.Ticker("CLN22.NYM")
#oil.history(start=datetime.datetime(datetime.date.today().year,datetime.date.today().month-2,1))

oil = get_yahoo_data(ticker="CLN22.NYM", start=datetime.datetime(datetime.date.today().year,datetime.date.today().month-2,1), end=datetime.datetime.now())
oil
# ticker = oil_futures_ticker_generator(year='22',month='jul')
# hist['ticker']=ticker
# hist.head()
# hist.index

In [ ]:
datetime.datetime(datetime.date.today().year,datetime.date.today().month,1)
#datetime.datetime(datetime.date.today().year,datetime.date.today().month+3,1).strftime('%h').lower()
#datetime.date.today().strftime('%b').lower()
# datetime.datetime.now()
#datetime.datetime(datetime.date.today().year,datetime.date.today().month+2,1).strftime('%-m')

## Database connectivity

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)

    return conn

In [ ]:
conn = create_connection('futures.db')
cur = conn.cursor()
cur.execute("SELECT TICKER, MAX(DATE) FROM OIL_FUTURES_YAHOO WHERE TICKER = 'CLU22.NYM' GROUP BY TICKER")
res = cur.fetchone()
print(type(res))
print(res)
print(type(res[0][1]))
datetime_obj = datetime.datetime.strptime(res[1], '%Y-%m-%d')
print(datetime_obj)
print(type(datetime_obj))
if datetime_obj <= datetime.datetime.now():
    print('True')
else:
    print('False')


In [ ]:
def create_entry(conn, row):
    """
    Create a new futures price entry in the database's OIL_FUTURES_YAHOO table
    :param conn:
    :param row:
    :return: void
    """
    sql = '''INSERT INTO OIL_FUTURES_YAHOO(TICKER, DATE, OPEN, HIGH, LOW, CLOSE, VOLUME)
            VALUES(?,?,?,?,?,?,?)'''
    try:
        cur = conn.cursor()
        cur.execute(sql,row)
        conn.commit()
    except Exception as e:
        print(e)

In [ ]:
for index,row in back_month_data.iterrows():
    print((row[5],index.date(),row[0],row[1],row[2],row[3],row[4]))
    row = (row[5],index.date(),row[0],row[1],row[2],row[3],row[4])
    create_entry(conn, row)


## Creating a class

In [1]:
import yfinance as yf
import sqlite3
#from yf_interface.base import yahoo_futures_interface
from configparser import ConfigParser
import datetime

In [8]:
#SETUP CONFIG OBJECT
config_object = ConfigParser()
config_object.read("config.ini")
server_config = config_object["SERVERINFO"]



month_symbol_mapper = {
    '1':{'string':'jan','code':'F'},
    '2':{'string':'feb','code':'G'},
    '3':{'string':'mar','code':'H'},
    '4':{'string':'apr','code':'J'},
    '5':{'string':'may','code':'K'},
    '6':{'string':'jun','code':'M'},
    '7':{'string':'jul','code':'N'},
    '8':{'string':'aug','code':'Q'},
    '9':{'string':'sep','code':'U'},
    '10':{'string':'oct','code':'V'},
    '11':{'string':'nov','code':'X'},
    '12':{'string':'dev','code':'Z'},
}

class yahoo_futures_interface:
    def __init__(self, year='22', month='1'):
        self.year = year
        self.month = month

    def _oil_futures_ticker_generator(self):
        self.ticker = f"CL{month_symbol_mapper[self.month]['code']}{self.year}.NYM"
        print(self.ticker)

    def get_yahoo_data(self, start=None):
        self._oil_futures_ticker_generator()
        self._get_start_date()
        yahoo_ticker_interface = yf.Ticker(self.ticker)
        self.price_data = yahoo_ticker_interface.history(
                start=self.start_date, end=datetime.datetime.now())
        self.price_data.drop(
            labels=['Dividends', 'Stock Splits'], axis=1, inplace=True)
        self.price_data['Ticker'] = self.ticker
    
    def _get_start_date(self):
        self._create_connection()
        self.cur.execute(f"SELECT MAX(DATE) FROM OIL_FUTURES_YAHOO WHERE TICKER = '{self.ticker+'i'}';")
        res = self.cur.fetchone()
        self.conn.close()
        print(res)
        if res[0] is not None :
            self.start_date = datetime.datetime.strptime(res[0], '%Y-%m-%d')
        else:
            self.start_date = datetime.datetime.now() - datetime.timedelta(45)
        print(self.start_date)


    def _create_connection(self, db_file=server_config['database']):
        """ create a database connection to the SQLite database
            specified by db_file
        :param db_file: database file
        :return: Connection object or None
        """
        self.conn = None
        try:
            self.conn = sqlite3.connect(db_file)
            self.cur = self.conn.cursor()
        except Exception as e:
            print(e)
    
    
    def _create_execute_entry(self, row):
        """
        Create a new futures price entry in the database's OIL_FUTURES_YAHOO table
        :param conn:
        :param row:
        :return: void
        """
        
        sql = '''INSERT INTO OIL_FUTURES_YAHOO(TICKER, DATE, OPEN, HIGH, LOW, CLOSE, VOLUME)
                VALUES(?,?,?,?,?,?,?)'''
        try:
            self._create_connection()
            self.cur.execute(sql,row)
            self.conn.commit()
            self.conn.close()
        except Exception as e:
            print(e)
    
    def create_entries_from_dataframe(self):
        #self._create_connection()
        for index,row in self.price_data.iterrows():
            print((row[5],index.date(),row[0],row[1],row[2],row[3],row[4]))
            row = (row[5],index.date(),row[0],row[1],row[2],row[3],row[4])
            self._create_execute_entry(row)
        


In [9]:
# sep = yahoo_futures_interface(year='22', month='9')
# sep.get_yahoo_data()
# sep.price_data
# sep.create_entries_from_dataframe()

In [10]:
def get_next_12_contract_months(month_symbol_mapper):
    year  = datetime.date.today().year
    month = datetime.date.today().month +2
    next_12_contracts = []
    step=0
    while step < 12:
        if month < 12:
            next_12_contracts.append({'year':str(year), 'month':str(month)})
            month +=1
        else:
            year+=1
            month = 1
            next_12_contracts.append({'year':str(year), 'month':str(month)})
        step+=1
    return next_12_contracts


next_12_contracts = get_next_12_contract_months(month_symbol_mapper)

In [11]:
for contract in next_12_contracts:
    print(f"year: {contract['year']}, month: {contract['month']}")
    interface = yahoo_futures_interface(year='22', month='9')
    interface.get_yahoo_data()
    interface.create_entries_from_dataframe()

year: 2022, month: 7
CLU22.NYM
(None,)
2022-04-09 16:42:27.763051
('CLU22.NYM', datetime.date(2022, 4, 11), 94.73999786376953, 94.73999786376953, 91.0999984741211, 92.45999908447266, 50437)
database is locked
('CLU22.NYM', datetime.date(2022, 4, 12), 93.41999816894531, 97.9000015258789, 92.98999786376953, 97.58000183105469, 47674)
database is locked
('CLU22.NYM', datetime.date(2022, 4, 13), 97.83000183105469, 100.66000366210938, 96.9800033569336, 100.5199966430664, 48784)


KeyboardInterrupt: 

: 

In [ ]:
# sep.most_recent_date
# yahoo_ticker_interface = yf.Ticker('CLU22.NYM')
# yahoo_ticker_interface.history(start=sep.most_recent_date)